In [1]:
import tensorflow as tf
import json
import os
import sys
from pathlib import Path
import keras

In [2]:
model = keras.models.Sequential([
    keras.layers.Conv2D(1,(3, 3),strides=(1,1), activation='relu', padding='valid',input_shape=(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.save("model1.keras")
model.summary()

e:\Apps_and_Programs\anaconda\envs\CNN_TF_GPU\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 1)      │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 169)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        10,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,300 (51.95 KB)

 Trainable params: 13,300 (51.95 KB)

 Non-trainable params: 0 (0.00 B)

### Stage 1

In [3]:
# stage1_sw_gen.py
"""
Stage 1: TensorFlow/Keras model → software_model.json
---------------------------------
Generates a hardware-friendly JSON description of the network.
"""


# ---------------- CONFIG ---------------- #
SUPPORTED_LAYERS = {
    "Conv2D": {
        "supported": ["filters", "kernel_size", "strides", "activation"],
        "valid_activations": ["relu", None],
    },
    "MaxPooling2D": {
        "supported": ["pool_size", "strides"]
    },
    "Dense": {
        "supported": ["units", "activation"],
        "valid_activations": ["relu", "sigmoid", "softmax"],
    },
    "Flatten": {"supported": []},
}

# ---------------- UTILITY FUNCTIONS ---------------- #

def _shape_to_list(shape):
    """Converts TensorShape/tuple to a clean Python list."""
    if shape is None:
        return None
    return [int(s) if s is not None else None for s in shape]

def _scalarize(param):
    """Convert tuple/list params like (3,3) -> 3 (requires square kernels)."""
    if isinstance(param, (list, tuple)):
        if len(set(param)) == 1:
            return param[0]
        raise ValueError(f"⚠️ Only square kernels supported, got {param}")
    return param

def _infer_layer_shapes(model):
    """Run dummy inference to infer input/output tensor shapes per layer."""
    if hasattr(model, "inputs") and model.inputs:
        input_shape = [1 if s is None else s for s in model.inputs[0].shape]
    elif hasattr(model, "input_shape") and model.input_shape:
        input_shape = [1 if s is None else s for s in model.input_shape]
    else:
        raise ValueError("❌ Could not infer model input shape.")

    dummy_input = tf.zeros(input_shape)
    x = dummy_input
    layer_shapes = []
    for layer in model.layers:
        in_shape = _shape_to_list(x.shape)
        x = layer(x)
        out_shape = _shape_to_list(x.shape)
        layer_shapes.append((in_shape, out_shape))
    return layer_shapes

def _extract_layer_info(layer, in_shape, out_shape):
    """Extracts simplified hardware-useful info per layer."""
    ltype = layer.__class__.__name__
    cfg = layer.get_config()
    data = {"name": layer.name, "type": ltype}

    # ---------------- Shapes ---------------- #
    if len(in_shape) == 4:
        _, in_h, in_w, in_c = in_shape
        data["input_shape"] = [in_h, in_w]
        data["input_channels"] = in_c
    elif len(in_shape) == 2:
        _, n = in_shape
        data["input_shape"] = [n]
        data["input_channels"] = 1

    if len(out_shape) == 4:
        _, out_h, out_w, out_c = out_shape
        data["output_shape"] = [out_h, out_w]
        data["output_channels"] = out_c
    elif len(out_shape) == 2:
        _, n = out_shape
        data["output_shape"] = [n]
        data["output_channels"] = n

    # ---------------- Parameters ---------------- #
    for key in SUPPORTED_LAYERS.get(ltype, {}).get("supported", []):
        if key in cfg:
            val = cfg[key]
            if key == "strides":
                data["stride"] = _scalarize(val)
            elif key == "kernel_size":
                data["kernel_size"] = _scalarize(val)
            elif key == "pool_size":
                data["pool_size"] = _scalarize(val)
            else:
                data[key] = val

    # ---------------- Activation ---------------- #
    act = cfg.get("activation", None)
    valid_acts = SUPPORTED_LAYERS.get(ltype, {}).get("valid_activations", [])
    if act and act not in valid_acts:
        raise ValueError(f"Unsupported activation '{act}' for layer {layer.name}")
    if act:
        data["activation"] = act

    return data

# ---------------- MAIN FUNCTION ---------------- #

def generate_software_json(model_path, out_path="software_model.json"):
    """Generates the software_model.json from a Keras model file."""
    print(f"📥 Loading Keras model: {model_path}")
    model = tf.keras.models.load_model(model_path, compile=False)

    print(f"⚙️ Inferring layer shapes...")
    layer_shapes = _infer_layer_shapes(model)

    print(f"🧩 Extracting layer configurations...")
    layers_out = []
    for layer, (in_s, out_s) in zip(model.layers, layer_shapes):
        ltype = layer.__class__.__name__
        if ltype not in SUPPORTED_LAYERS:
            print(f"⚠️ Skipping unsupported layer type: {ltype}")
            continue
        info = _extract_layer_info(layer, in_s, out_s)
        layers_out.append(info)

    out_json = {
        "model_name": model.name,
        "layers": layers_out
    }

    Path(out_path).write_text(json.dumps(out_json, indent=2))
    print(f"✅ Software model JSON written to {out_path}")
    return out_json

# ---------------- Example Run ---------------- #
if __name__ == "__main__":
    generate_software_json("model1.keras", "software_model.json")


📥 Loading Keras model: model1.keras
⚙️ Inferring layer shapes...
🧩 Extracting layer configurations...
✅ Software model JSON written to software_model.json


### Stage 2 

In [7]:
# stage2_hw_gen.py
"""
Stage 2: software_model.json → hardware_model.json
--------------------------------------------------
Generates hardware-level JSON from software description JSON.

✓ Separate counters for Conv / Pool / NN layers
✓ Conv MIF names match 'Memory_Files/mif' convention
✓ Neural network section includes full layer information
✓ NN file structure consistent with per-neuron MIF generation
"""

import json
from pathlib import Path


# --------------------------- CONFIG --------------------------- #

GLOBAL_DEFAULTS = {
    "DATA_WIDTH": 16,
    "FRACTION_SIZE": 14,
    "SIGNED": 1,
    "GUARD_TYPE": 2
}


# --------------------------- BLOCK BUILDERS --------------------------- #

def make_imgbuf_block(name, layer, prev_out, stride, channels, kernel_size):
    """Create ImageBufferChnl block JSON."""
    return {
        name: {
            "module": "ImageBufferChnl",
            "parameters": {
                "KERNEL_SIZE": kernel_size,
                "DATA_WIDTH": "DATA_WIDTH",
                "COLUMN_NUM": layer["input_shape"][0],
                "ROW_NUM": layer["input_shape"][1],
                "STRIDE": stride,
                "CHANNELS": channels
            },
            "inputs": {
                "clock": {"name": "clock", "width": 1},
                "sreset_n": {"name": "sreset_n", "width": 1},
                "data_valid": {"name": "data_valid" if prev_out == "data_in" else f"{prev_out}_valid", "width": 1},
                "data_in": {"name": prev_out, "width": "DATA_WIDTH"}
            },
            "outputs": {
                "data_out": {"name": f"{name}_out", "width": "DATA_WIDTH"},
                "kernel_out": {"name": f"kernel_out_{name}", "width": f"DATA_WIDTH * {kernel_size} * {kernel_size}"},
                "kernel_valid": {"name": f"kernel_valid_{name}", "width": 1}
            }
        }
    }


def make_conv_block(name, layer, ibuf_name, conv_block_num):
    """Create ConvChnl block JSON with correct MIF filenames."""
    k = layer["kernel_size"]
    return {
        name: {
            "module": "ConvChnl",
            "parameters": {
                "KERNEL_SIZE": k,
                "CHANNELS": layer["input_channels"],
                "DATA_WIDTH": "DATA_WIDTH",
                "FRACTION_SIZE": "FRACTION_SIZE",
                "SIGNED": "SIGNED",
                "ACTIVATION": layer.get("activation", "none"),
                "GUARD_TYPE": "GUARD_TYPE"
            },
            "files": {
                "weights_file": f"c_{conv_block_num}_w.mif",
                "biases_file": f"c_{conv_block_num}_b.mif"
            },
            "inputs": {
                "clock": {"name": "clock", "width": 1},
                "sreset_n": {"name": "sreset_n", "width": 1},
                "data_valid": {"name": f"kernel_valid_{ibuf_name}", "width": 1},
                "kernel_in": {"name": f"kernel_out_{ibuf_name}", "width": f"DATA_WIDTH * {k} * {k}"}
            },
            "outputs": {
                "conv_out": {"name": f"{name}_out", "width": "DATA_WIDTH"},
                "conv_valid": {"name": f"{name}_valid", "width": 1}
            }
        }
    }


def make_maxpool_block(name, layer, ibuf_name):
    """Create MaxpoolChnl block JSON."""
    k = layer["pool_size"]
    return {
        name: {
            "module": "MaxpoolChnl",
            "parameters": {
                "KERNEL_SIZE": k,
                "DATA_WIDTH": "DATA_WIDTH",
                "SIGNED": "SIGNED",
                "CHANNELS": layer["input_channels"]
            },
            "inputs": {
                "clock": {"name": "clock", "width": 1},
                "sreset_n": {"name": "sreset_n", "width": 1},
                "data_valid": {"name": f"kernel_valid_{ibuf_name}", "width": 1},
                "kernel_in": {"name": f"kernel_out_{ibuf_name}", "width": f"DATA_WIDTH * {k} * {k}"}
            },
            "outputs": {
                "maxp_out": {"name": f"{name}_out", "width": "DATA_WIDTH"},
                "maxp_valid": {"name": f"{name}_valid", "width": 1}
            }
        }
    }


def make_maxfinder_block(prev_out_signal, num_outputs):
    """Create maxFinder block as final classifier stage."""
    return {
        "maxFinder": {
            "module": "maxFinder",
            "parameters": {
                "NUM_INPUTS": num_outputs,
                "DATA_WIDTH": "DATA_WIDTH",
                "SIGNED": "SIGNED"
            },
            "inputs": {
                "clock": {"name": "clock", "width": 1},
                "sreset_n": {"name": "sreset_n", "width": 1},
                "data_valid": {"name": f"{prev_out_signal}_valid", "width": 1},
                "data_in": {"name": prev_out_signal, "width": f"DATA_WIDTH * {num_outputs}"}
            },
            "outputs": {
                "class_idx": {"name": "class_idx", "width": "log2(NUM_INPUTS)"},
                "valid": {"name": "class_valid", "width": 1}
            }
        }
    }


def build_nn_block(prev_out_signal, dense_layers):
    """
    Build NN structure describing Dense layers.
    File naming remains consistent with neuron-per-MIF generation.
    """
    nn_layers = []
    for i, layer in enumerate(dense_layers, start=1):
        entry = {
            "name": f"dense_L{i}",
            "num_inputs": layer["input_shape"][0],
            "num_neurons": layer["units"],
            "activation": layer.get("activation", "none"),
            # NOTE: MIF file naming convention placeholders
            "weight_file_pattern": f"w_{i}_<neuron>.mif",
            "bias_file_pattern": f"b_{i}_<neuron>.mif"
        }
        nn_layers.append(entry)

    return {
        "Build": True,
        "globals": {
            "DATA_WIDTH": "DATA_WIDTH",
            "FRACTION_SIZE": "FRACTION_SIZE",
            "SIGNED": "SIGNED"
        },
        "io": {
            "input_signal": prev_out_signal,
            "input_valid": f"{prev_out_signal}_valid",
            "output_signal": "nn_out",
            "output_valid": "nn_valid"
        },
        "layers": nn_layers
    }


# --------------------------- MAIN CONVERSION --------------------------- #

def generate_hardware_json(input_file="software_model.json", output_file="hardware_model.json"):
    """Main Stage 2 entry point."""
    print(f"📄 Reading {input_file} ...")
    with open(input_file, "r") as f:
        model = json.load(f)

    layers = model["layers"]
    globals_ = GLOBAL_DEFAULTS.copy()

    top = {
        "module": "CNN",
        "hardware parameters": globals_,
        "inputs": {
            "clock": 1,
            "sreset_n": 1,
            "data_in": "DATA_WIDTH",
            "data_valid": 1
        },
        "outputs": {
            "class_idx": "log2(NUM_INPUTS)",
            "class_valid": 1
        }
    }

    # Independent counters
    conv_idx = 1
    pool_idx = 1
    dense_layers = []
    prev_out = "data_in"

    for layer in layers:
        ltype = layer["type"]

        if ltype == "Conv2D":
            ibuf_name = f"ImageBuffer_Conv{conv_idx}"
            conv_name = f"Conv{conv_idx}"

            top.update(make_imgbuf_block(
                ibuf_name, layer, prev_out,
                stride=layer["stride"],
                channels=layer["input_channels"],
                kernel_size=layer["kernel_size"]
            ))
            top.update(make_conv_block(conv_name, layer, ibuf_name, conv_idx))

            prev_out = f"{conv_name}_out"
            conv_idx += 1

        elif ltype == "MaxPooling2D":
            ibuf_name = f"ImageBuffer_Maxpool{pool_idx}"
            pool_name = f"Maxpool{pool_idx}"

            top.update(make_imgbuf_block(
                ibuf_name, layer, prev_out,
                stride=layer["stride"],
                channels=layer["input_channels"],
                kernel_size=layer["pool_size"]
            ))
            top.update(make_maxpool_block(pool_name, layer, ibuf_name))

            prev_out = f"{pool_name}_out"
            pool_idx += 1

        elif ltype == "Flatten":
            continue

        elif ltype == "Dense":
            dense_layers.append(layer)

    # Add NN and MaxFinder
    if dense_layers:
        top["NN"] = build_nn_block(prev_out, dense_layers)
        last_layer = dense_layers[-1]
        num_outputs = last_layer["units"]
        top.update(make_maxfinder_block("nn_out", num_outputs))
    else:
        top["outputs"] = {
            "data_out": prev_out,
            "data_valid": f"{prev_out}_valid"
        }

    final_json = {"Hardware_model": {"top": top}}

    Path(output_file).write_text(json.dumps(final_json, indent=2))
    print(f"✅ Hardware model written to {output_file}")

if __name__ == "__main__":
    generate_hardware_json("software_model.json", "hardware_model.json")

📄 Reading software_model.json ...
✅ Hardware model written to hardware_model.json


### MIF Generation

In [1]:
# mif_writer.py
"""
MIF Writer Utility (Final Stable Version)
-----------------------------------------
Generates .mif (binary) and .csv (float) files for Conv2D and Dense (neuron)
layers, and optionally saves a Memory_Index.json describing all files.

Features:
---------
• Writes binary data in .mif (ready for $readmemb).
• Cleans previous output folder automatically.
• Uses default output folder 'Memory_Files' if none provided.
• Optionally saves a JSON index file when save_index=True.
• Returns nothing (pure side-effect operation).
"""

import shutil
import json
import numpy as np
from pathlib import Path


# ------------------ Fixed-Point Conversion ------------------ #
def quantize_to_fixed(value, fraction_bits, data_width, signed=True):
    """Convert float → fixed-point integer (two’s complement)."""
    scaled = value * (2 ** fraction_bits)
    q = int(round(scaled))

    if signed:
        max_val = 2 ** (data_width - 1) - 1
        min_val = -(2 ** (data_width - 1))
        q = max(min(q, max_val), min_val)
        if q < 0:
            q = (1 << data_width) + q  # two's complement
    else:
        max_val = 2 ** data_width - 1
        q = max(min(q, max_val), 0)

    return q


def int_to_bin(value, data_width):
    """Convert integer to zero-padded binary string."""
    return format(value, f"0{data_width}b")


def write_list(filepath, values, data_width, fmt="mif"):
    """Write values to file (.mif for binary, .csv for floats)."""
    Path(filepath).parent.mkdir(exist_ok=True, parents=True)
    with open(filepath, "w") as f:
        if fmt == "mif":
            for val in values:
                f.write(int_to_bin(val, data_width) + "\n")
        elif fmt == "csv":
            for val in values:
                f.write(f"{val}\n")


# ------------------ Conv2D Export ------------------ #
def generate_conv_mif_files(weights, biases, conv_block_num, data_width, fraction_bits, signed, base_dir):
    """Generate .mif/.csv files for a Conv2D layer."""
    w = np.array(weights)
    b = np.array(biases)
    k_h, k_w, in_ch, out_ch = w.shape
    ordered_values = []

    # Order: OC major → IC → kernel (kernel position fastest)
    for oc in range(out_ch):
        for ic in range(in_ch):
            for kh in range(k_h):
                for kw in range(k_w):
                    ordered_values.append(float(w[kh, kw, ic, oc]))

    q_weights = [quantize_to_fixed(v, fraction_bits, data_width, signed) for v in ordered_values]
    q_biases = [quantize_to_fixed(float(x), fraction_bits, data_width, signed) for x in b]

    mif_dir = Path(base_dir) / "mif"
    csv_dir = Path(base_dir) / "csv"

    # File paths
    w_mif = mif_dir / f"c_{conv_block_num}_w.mif"
    b_mif = mif_dir / f"c_{conv_block_num}_b.mif"
    w_csv = csv_dir / f"c_{conv_block_num}_w.csv"
    b_csv = csv_dir / f"c_{conv_block_num}_b.csv"

    write_list(w_mif, q_weights, data_width, fmt="mif")
    write_list(b_mif, q_biases, data_width, fmt="mif")
    write_list(w_csv, ordered_values, data_width, fmt="csv")
    write_list(b_csv, b.tolist(), data_width, fmt="csv")

    print(f"✅ Conv block {conv_block_num}: {len(q_weights)} weights, {len(q_biases)} biases")

    return {
        "conv_block": conv_block_num,
        "weights_mif": str(w_mif),
        "biases_mif": str(b_mif),
        "weights_csv": str(w_csv),
        "biases_csv": str(b_csv)
    }


# ------------------ Dense Export (Per Neuron) ------------------ #
def generate_dense_mif_files(weights, biases, layer_number, data_width, fraction_bits, signed, base_dir):
    """Generate per-neuron .mif/.csv files for one Dense layer."""
    w = np.array(weights)
    b = np.array(biases)
    in_dim, out_dim = w.shape

    mif_dir = Path(base_dir) / "mif"
    csv_dir = Path(base_dir) / "csv"

    neuron_files = []
    for neuron in range(out_dim):
        weights_list = [float(w[inp, neuron]) for inp in range(in_dim)]
        bias_val = float(b[neuron])

        q_weights = [quantize_to_fixed(v, fraction_bits, data_width, signed) for v in weights_list]
        q_bias = quantize_to_fixed(bias_val, fraction_bits, data_width, signed)

        # File paths
        w_mif = mif_dir / f"w_{layer_number}_{neuron}.mif"
        b_mif = mif_dir / f"b_{layer_number}_{neuron}.mif"
        w_csv = csv_dir / f"w_{layer_number}_{neuron}.csv"
        b_csv = csv_dir / f"b_{layer_number}_{neuron}.csv"

        write_list(w_mif, q_weights, data_width, fmt="mif")
        write_list(b_mif, [q_bias], data_width, fmt="mif")
        write_list(w_csv, weights_list, data_width, fmt="csv")
        write_list(b_csv, [bias_val], data_width, fmt="csv")

        neuron_files.append({
            "neuron": neuron,
            "weights_mif": str(w_mif),
            "bias_mif": str(b_mif),
            "weights_csv": str(w_csv),
            "bias_csv": str(b_csv)
        })

        print(f"✅ Neuron {neuron} in layer {layer_number}: "
              f"{len(weights_list)} weights, 1 bias")

    return {"layer": layer_number, "neurons": neuron_files}


# ------------------ Master Export ------------------ #
def export_mifs_from_keras(model_path,
                           data_width=16,
                           fraction_bits=14,
                           signed=True,
                           base_dir=None,
                           save_index=True):
    """
    Generates .mif (binary) and .csv (float) files for Conv2D and Dense layers.
    Cleans output folder each run. Optionally saves Memory_Index.json.
    Returns nothing.
    """
    import tensorflow as tf
    model = tf.keras.models.load_model(model_path, compile=False)

    # Default folder
    if base_dir is None:
        base_dir = "Memory_Files"

    base_dir = Path(base_dir)

    # Clean existing folder
    if base_dir.exists():
        shutil.rmtree(base_dir)
        print(f"🧹 Removed existing folder: {base_dir}")

    (base_dir / "mif").mkdir(parents=True, exist_ok=True)
    (base_dir / "csv").mkdir(parents=True, exist_ok=True)

    conv_block_num = 1
    nn_layer_num = 1
    generated = {"conv_blocks": [], "nn_layers": []}

    # Generate MIFs
    for layer in model.layers:
        ltype = layer.__class__.__name__

        if ltype == "Conv2D":
            w, b = layer.get_weights()
            files = generate_conv_mif_files(w, b, conv_block_num, data_width, fraction_bits, signed, base_dir)
            generated["conv_blocks"].append(files)
            conv_block_num += 1

        elif ltype == "Dense":
            w, b = layer.get_weights()
            files = generate_dense_mif_files(w, b, nn_layer_num, data_width, fraction_bits, signed, base_dir)
            generated["nn_layers"].append(files)
            nn_layer_num += 1

        else:
            continue

    # Optional JSON index
    if save_index:
        index_path = base_dir / "Memory_Index.json"
        with open(index_path, "w") as f:
            json.dump(generated, f, indent=2)
        print(f"🗂️  Saved index file: {index_path}")

    print(f"\n📁 All files generated under: {base_dir.resolve()}")
    # No return value (intentional)
    return


if __name__ == "__main__":
    export_mifs_from_keras(
    model_path="model.keras",
    data_width=16,
    fraction_bits=14,
    signed=True,
    save_index=True   # set False to skip Memory_Index.json
)

🧹 Removed existing folder: Memory_Files
✅ Conv block 1: 9 weights, 1 biases
✅ Neuron 0 in layer 1: 169 weights, 1 bias
✅ Neuron 1 in layer 1: 169 weights, 1 bias
✅ Neuron 2 in layer 1: 169 weights, 1 bias
✅ Neuron 3 in layer 1: 169 weights, 1 bias
✅ Neuron 4 in layer 1: 169 weights, 1 bias
✅ Neuron 5 in layer 1: 169 weights, 1 bias
✅ Neuron 6 in layer 1: 169 weights, 1 bias
✅ Neuron 7 in layer 1: 169 weights, 1 bias
✅ Neuron 8 in layer 1: 169 weights, 1 bias
✅ Neuron 9 in layer 1: 169 weights, 1 bias
✅ Neuron 10 in layer 1: 169 weights, 1 bias
✅ Neuron 11 in layer 1: 169 weights, 1 bias
✅ Neuron 12 in layer 1: 169 weights, 1 bias
✅ Neuron 13 in layer 1: 169 weights, 1 bias
✅ Neuron 14 in layer 1: 169 weights, 1 bias
✅ Neuron 15 in layer 1: 169 weights, 1 bias
✅ Neuron 16 in layer 1: 169 weights, 1 bias
✅ Neuron 17 in layer 1: 169 weights, 1 bias
✅ Neuron 18 in layer 1: 169 weights, 1 bias
✅ Neuron 19 in layer 1: 169 weights, 1 bias
✅ Neuron 20 in layer 1: 169 weights, 1 bias
✅ Neuron 2

In [39]:
#!/usr/bin/env python3
"""
build_rtl_v5.py

Fully fixed & JSON-driven RTL builder:
 - copies designFiles/*.v -> rtl/
 - generates rtl/CNN_top.v (readable) and rtl/Layer_X.v files
 - order-agnostic: detects which block precedes NN
 - uses JSON-defined output names for Conv2D / Max2D (no _conv_out/_maxp_out suffixes)
 - generates FSMs between NN layers, and final holdData + maxFinder for softmax/hardmax
 - enforces mandatory parameters and strict validation
"""

import json
import shutil
from pathlib import Path
import os

# ---------- Config ----------
INPUT_JSON = "hardware_model.json"
DESIGN_DIR = Path("designFiles")
RTL_DIR = Path("rtl")
VERBOSE = True

DEPENDENCY_BASE = {
    "Conv2D": ["FP_Adder.v", "FP_Multiplier.v", "ConvMemory.v", "Conv_SIC.v", "Conv_MIC.v", "relu.v"],
    "Max2D": ["FP_Comparator.v", "Maxpool.v"],
    "neuron": ["FP_Adder.v", "FP_Multiplier.v", "Weight_Memory.v", "Sig_ROM.v", "relu.v"],
    "maxFinder": []
}

# ---------- Utils ----------
def vprint(*a):
    if VERBOSE:
        print(*a)

def ensure_dirs():
    RTL_DIR.mkdir(parents=True, exist_ok=True)

def read_json():
    p = Path(INPUT_JSON)
    if not p.exists():
        raise FileNotFoundError(f"{INPUT_JSON} not found in cwd={os.getcwd()}")
    with open(p, "r") as f:
        d = json.load(f)
    if "Hardware_model" not in d or "top" not in d["Hardware_model"]:
        raise KeyError("hardware_model.json must have Hardware_model.top")
    return d["Hardware_model"]["top"]

def copy_if_exists(fname):
    src = DESIGN_DIR / fname
    dst = RTL_DIR / fname
    if not src.exists():
        vprint(f"⚠ Missing design file: {src}")
        return False
    shutil.copy2(src, dst)
    vprint(f"→ Copied {src.name} -> rtl/")
    return True

def resolve_and_copy(mod):
    cand = [f"{mod}.v"]
    if mod.lower() in ("conv2d", "convch", "convolchnl", "convchnl"):
        cand += ["Conv2D.v", "ConvolChnl.v", "ConvChnl.v"]
    found = False
    for c in cand:
        if (DESIGN_DIR / c).exists():
            copy_if_exists(c)
            found = True
            break
    if not found:
        vprint(f"⚠ Could not locate module {mod} (tried {cand})")
    for dep in DEPENDENCY_BASE.get(mod, []):
        copy_if_exists(dep)

def ensure_keys(d, keys, ctx):
    for k in keys:
        if k not in d:
            raise KeyError(f"[{ctx}] Missing mandatory parameter: '{k}'")

def derive_prev_output_channels(top):
    """Find block before NN and derive its output channel count."""
    keys = list(top.keys())
    try:
        idx = keys.index("NN")
    except ValueError:
        raise KeyError("NN block missing in JSON")
    for k in reversed(keys[:idx]):
        blk = top[k]
        if isinstance(blk, dict) and "module" in blk:
            params = blk.get("parameters", {})
            if "OUTPUT_CHANNELS" in params:
                return int(params["OUTPUT_CHANNELS"])
            if "CHANNELS" in params:
                return int(params["CHANNELS"])
    raise KeyError("Could not derive output channels from previous block")

def find_block_before_nn(top):
    keys = list(top.keys())
    try:
        idx = keys.index("NN")
    except ValueError:
        raise KeyError("NN block missing in JSON")
    for k in reversed(keys[:idx]):
        blk = top[k]
        if isinstance(blk, dict) and "module" in blk:
            return k, blk
    raise KeyError("No valid block found before NN")

# ---------- Layer File Generator ----------
def gen_layer_file(idx, layer, hw_params, first_layer_input_channels=None):
    name = f"Layer_{idx}"
    DW = int(hw_params["DATA_WIDTH"])
    FRAC = int(hw_params["FRACTION_SIZE"])
    weightIntWidth = DW - FRAC
    SIGMOID_SIZE = hw_params.get("SIGMOID_SIZE", 10)

    if "input_channels" in layer:
        in_ch = int(layer["input_channels"])
    elif idx == 1 and first_layer_input_channels:
        in_ch = first_layer_input_channels
    else:
        in_ch = 1

    NN = int(layer["num_neurons"])
    numW = int(layer["num_inputs"])
    act = layer.get("activation", "relu")
    wpat = layer.get("weight_file_pattern", f"w_{idx}_<neuron>.mif")
    bpat = layer.get("bias_file_pattern", f"b_{idx}_<neuron>.mif")

    lines = [
        "// =============================================================",
        f"// {name} — {layer.get('name','layer')}, act={act}, neurons={NN}",
        "// =============================================================",
        f"module {name} #(",
        f"    parameter NN = {NN},",
        f"    parameter numWeight = {numW},",
        f"    parameter dataWidth = {DW},",
        f"    parameter layerNum = {idx},",
        f"    parameter sigmoidSize = {SIGMOID_SIZE},",
        f"    parameter weightIntWidth = {weightIntWidth},",
        f"    parameter input_channels = {in_ch},",
        f"    parameter actType = \"{act}\"",
        f")(",
        f"    input           clk,",
        f"    input           rst,",
        f"    input           weightValid,",
        f"    input           biasValid,",
        f"    input  [31:0]   weightValue,",
        f"    input  [31:0]   biasValue,",
        f"    input  [31:0]   config_layer_num,",
        f"    input  [31:0]   config_neuron_num,",
        f"    input           x_valid,",
        f"    input  [input_channels*dataWidth-1:0] x_in,",
        f"    output [NN-1:0] o_valid,",
        f"    output [NN*dataWidth-1:0] x_out",
        f");",
        ""
    ]

    for n in range(NN):
        wfile = wpat.replace("<neuron>", str(n))
        bfile = bpat.replace("<neuron>", str(n))
        lines += [
            f"    neuron #(",
            f"        .input_channels(input_channels),",
            f"        .numWeight(numWeight),",
            f"        .layerNo({idx}),",
            f"        .neuronNo({n}),",
            f"        .dataWidth(dataWidth),",
            f"        .sigmoidSize(sigmoidSize),",
            f"        .weightIntWidth(weightIntWidth),",
            f"        .actType(actType),",
            f"        .weightFile(\"{wfile}\"),",
            f"        .biasFile(\"{bfile}\")",
            f"    ) n_{n} (",
            f"        .clk(clk),",
            f"        .rst(rst),",
            f"        .myinput(x_in),",
            f"        .weightValid(weightValid),",
            f"        .biasValid(biasValid),",
            f"        .weightValue(weightValue),",
            f"        .biasValue(biasValue),",
            f"        .config_layer_num(config_layer_num),",
            f"        .config_neuron_num(config_neuron_num),",
            f"        .myinputValid(x_valid),",
            f"        .out(x_out[{n}*dataWidth +: dataWidth]),",
            f"        .outvalid(o_valid[{n}])",
            f"    );",
            ""
        ]

    lines.append("endmodule")
    (RTL_DIR / f"{name}.v").write_text("\n".join(lines))
    vprint(f"✓ Wrote {name}.v")

# ---------- Top Generator ----------
def gen_top(top):
    hwp = top["hardware parameters"]
    ensure_keys(hwp, ["DATA_WIDTH","FRACTION_SIZE","SIGNED","GUARD_TYPE"], "hardware parameters")
    DW = int(hwp["DATA_WIDTH"])
    FRAC = int(hwp["FRACTION_SIZE"])

    lines = [
        "// =============================================================",
        "// Auto-generated CNN_top (readable, JSON-driven)",
        "// =============================================================",
        f"module CNN_top #(",
        f"    parameter DATA_WIDTH = {DW},",
        f"    parameter FRACTION_SIZE = {hwp['FRACTION_SIZE']},",
        f"    parameter SIGNED = {hwp['SIGNED']},",
        f"    parameter GUARD_TYPE = {hwp['GUARD_TYPE']}",
        f")(",
        f"    input wire clock,",
        f"    input wire sreset_n,",
        f"    input wire [DATA_WIDTH-1:0] data_in,",
        f"    input wire data_valid,",
        f"    output wire [3:0] class_idx,",
        f"    output wire class_valid",
        f");",
        "",
        "    wire reset = ~sreset_n;",
        ""
    ]

    prev_block_name = None
    prev_block = None
    for k, blk in top.items():
        if k in ("hardware parameters","inputs","outputs","module","NN","maxFinder"):
            continue
        if not isinstance(blk, dict) or "module" not in blk:
            continue

        mod = blk["module"]
        params = blk.get("parameters", {})
        inputs = blk.get("inputs", {})
        outputs = blk.get("outputs", {})

        # ---------- Conv2D ----------
        if mod.lower() == "conv2d":
            required_conv = [
                "KERNEL_SIZE","COLUMN_NUM","ROW_NUM","STRIDE",
                "INPUT_CHANNELS","OUTPUT_CHANNELS","DATA_WIDTH",
                "FRACTION_SIZE","SIGNED","ACTIVATION","GUARD_TYPE",
                "WEIGHT_FILE","BIAS_FILE"
            ]
            ensure_keys(params, required_conv, f"{k} Conv2D")
            out_ch = int(params["OUTPUT_CHANNELS"])

            conv_out_sig = outputs.get("conv_out", {}).get("name", f"{k}_out")
            conv_valid_sig = outputs.get("conv_valid", {}).get("name", f"{k}_valid")

            lines += [
                "    // ------------------------------------------------------------",
                f"    // {k}: Conv2D",
                "    // ------------------------------------------------------------",
                f"    wire [{out_ch}*DATA_WIDTH-1:0] {conv_out_sig};",
                f"    wire {conv_valid_sig};",
                "",
                f"    Conv2D #("
            ]
            for pname, pval in params.items():
                if pname in ("WEIGHT_FILE","BIAS_FILE"):
                    lines.append(f"        .{pname}(\"{pval}\"),")
                else:
                    lines.append(f"        .{pname}({pval}),")
            lines[-1] = lines[-1].rstrip(',')
            lines += [
                f"    ) {k} (",
                f"        .clock(clock),",
                f"        .sreset_n(sreset_n),",
                f"        .data_valid({inputs.get('data_valid',{}).get('name','data_valid')}),",
                f"        .data_in({inputs.get('data_in',{}).get('name','data_in')}),",
                f"        .conv_out({conv_out_sig}),",
                f"        .conv_valid({conv_valid_sig})",
                f"    );",
                ""
            ]
            resolve_and_copy("Conv2D")
            prev_block_name, prev_block = k, blk

        # ---------- Max2D ----------
        elif mod.lower() == "max2d":
            required_max = ["KERNEL_SIZE","DATA_WIDTH","COLUMN_NUM","ROW_NUM","STRIDE","CHANNELS","SIGNED"]
            ensure_keys(params, required_max, f"{k} Max2D")
            out_ch = int(params["CHANNELS"])

            max_out_sig = outputs.get("maxp_out", {}).get("name", f"{k}_out")
            max_valid_sig = outputs.get("maxp_valid", {}).get("name", f"{k}_valid")

            lines += [
                "    // ------------------------------------------------------------",
                f"    // {k}: Max2D",
                "    // ------------------------------------------------------------",
                f"    wire [{out_ch}*DATA_WIDTH-1:0] {max_out_sig};",
                f"    wire {max_valid_sig};",
                "",
                f"    Max2D #("
            ]
            for pname, pval in params.items():
                lines.append(f"        .{pname}({pval}),")
            lines[-1] = lines[-1].rstrip(',')
            lines += [
                f"    ) {k} (",
                f"        .clock(clock),",
                f"        .sreset_n(sreset_n),",
                f"        .data_valid({inputs.get('data_valid',{}).get('name','data_valid')}),",
                f"        .data_in({inputs.get('data_in',{}).get('name','data_in')}),",
                f"        .maxp_out({max_out_sig}),",
                f"        .maxp_valid({max_valid_sig})",
                f"    );",
                ""
            ]
            resolve_and_copy("Max2D")
            prev_block_name, prev_block = k, blk

    # ----------- Derive connection to NN -----------
    prev_outputs = prev_block["outputs"]
    prev_data_sig = list(prev_outputs.values())[0]["name"]
    prev_valid_sig = list(prev_outputs.values())[1]["name"]

    nn_block = top["NN"]
    layers = nn_block["layers"]
    first_in_ch = derive_prev_output_channels(top)

    # ----------- Generate Layer_X.v files -----------
    for i, layer in enumerate(layers, start=1):
        gen_layer_file(i, layer, hwp, first_layer_input_channels=first_in_ch if i == 1 else None)
        resolve_and_copy("neuron")

    # ----------- NN chaining + FSMs -----------
    lines += [
        "    // ------------------------------------------------------------",
        "    // Neural Network Layers (with IDLE/SEND FSMs between layers)",
        "    // ------------------------------------------------------------",
        "    localparam IDLE = 1'b0;",
        "    localparam SEND = 1'b1;",
        ""
    ]

    prev_data = prev_data_sig
    prev_valid = prev_valid_sig
    prev_num_neurons = None
    SIGMOID_SIZE = int(hwp.get("SIGMOID_SIZE", 10))
    weightIntWidth = DW - FRAC

    for i, layer in enumerate(layers, start=1):
        NN = int(layer["num_neurons"])
        numWeight = int(layer["num_inputs"])
        act = layer.get("activation", "relu")
        in_ch = int(layer.get("input_channels", prev_num_neurons or first_in_ch))
        prev_num_neurons = NN

        lines += [
            f"    // ------------------------------------------------------------",
            f"    // Layer {i} — {layer.get('name','layer')} ({act.upper()}, {NN} neurons)",
            f"    // ------------------------------------------------------------",
            f"    wire [{NN-1}:0] o{i}_valid;",
            f"    wire [{NN}*DATA_WIDTH-1:0] x{i}_out;",
            f"    reg  [{NN}*DATA_WIDTH-1:0] holdData_{i};",
            f"    reg  [DATA_WIDTH-1:0] out_data_{i};",
            f"    reg  data_out_valid_{i};",
            "",
            f"    Layer_{i} #(",
            f"        .NN({NN}),",
            f"        .numWeight({numWeight}),",
            f"        .dataWidth(DATA_WIDTH),",
            f"        .layerNum({i}),",
            f"        .sigmoidSize({SIGMOID_SIZE}),",
            f"        .weightIntWidth({weightIntWidth}),",
            f"        .input_channels({in_ch}),",
            f"        .actType(\"{act}\")",
            f"    ) L{i} (",
            f"        .clk(clock),",
            f"        .rst(reset),",
            f"        .weightValid(weightValid),",
            f"        .biasValid(biasValid),",
            f"        .weightValue(weightValue),",
            f"        .biasValue(biasValue),",
            f"        .config_layer_num(config_layer_num),",
            f"        .config_neuron_num(config_neuron_num),",
            f"        .x_valid({prev_valid}),",
            f"        .x_in({prev_data}),",
            f"        .o_valid(o{i}_valid),",
            f"        .x_out(x{i}_out)",
            f"    );",
            "",
            f"    reg state_{i}; integer count_{i};",
            f"    always @(posedge clock) begin",
            f"        if (reset) begin",
            f"            state_{i} <= IDLE; count_{i} <= 0; data_out_valid_{i} <= 0;",
            f"        end else case (state_{i})",
            f"            IDLE: begin",
            f"                count_{i} <= 0; data_out_valid_{i} <= 0;",
            f"                if (o{i}_valid[0]) begin holdData_{i} <= x{i}_out; state_{i} <= SEND; end",
            f"            end",
            f"            SEND: begin",
            f"                out_data_{i} <= holdData_{i}[DATA_WIDTH-1:0];",
            f"                holdData_{i} <= holdData_{i} >> DATA_WIDTH;",
            f"                count_{i} <= count_{i}+1; data_out_valid_{i} <= 1;",
            f"                if (count_{i} == {NN}) begin state_{i} <= IDLE; data_out_valid_{i} <= 0; end",
            f"            end",
            f"        endcase",
            f"    end",
            ""
        ]

        prev_data = f"out_data_{i}"
        prev_valid = f"data_out_valid_{i}"

    # ----------- Final Layer + maxFinder -----------
    last_layer = layers[-1]
    last_NN = last_layer["num_neurons"]
    last_act = last_layer["activation"].lower()

    if last_act in ("softmax","hardmax"):
        lines += [
            "    // ------------------------------------------------------------",
            "    // Final Layer + maxFinder",
            "    // ------------------------------------------------------------",
            f"    reg [{last_NN}*DATA_WIDTH-1:0] holdData_final;",
            f"    always @(posedge clock) begin",
            f"        if (o{len(layers)}_valid[0]) holdData_final <= x{len(layers)}_out;",
            f"    end",
            "",
            f"    maxFinder #(",
            f"        .NUM_INPUTS({last_NN}),",
            f"        .DATA_WIDTH(DATA_WIDTH),",
            f"        .SIGNED(SIGNED)",
            f"    ) mFind (",
            f"        .i_clk(clock),",
            f"        .i_data(x{len(layers)}_out),",
            f"        .i_valid(o{len(layers)}_valid),",
            f"        .o_data(class_idx),",
            f"        .o_data_valid(class_valid)",
            f"    );",
            ""
        ]
        resolve_and_copy("maxFinder")

    lines.append("endmodule")
    (RTL_DIR / "CNN_top.v").write_text("\n".join(lines))
    vprint("✓ Wrote rtl/CNN_top.v")

# ---------- Main ----------
def main():
    ensure_dirs()
    top = read_json()
    for mod in ["Conv2D", "Max2D", "neuron", "maxFinder"]:
        resolve_and_copy(mod)
    gen_top(top)
    vprint("\n✅ build_rtl_v5 complete. Inspect rtl/*.v\n")

if __name__ == "__main__":
    main()


→ Copied Conv2D.v -> rtl/
→ Copied FP_Adder.v -> rtl/
→ Copied FP_Multiplier.v -> rtl/
→ Copied ConvMemory.v -> rtl/
→ Copied Conv_SIC.v -> rtl/
→ Copied Conv_MIC.v -> rtl/
→ Copied relu.v -> rtl/
→ Copied Max2D.v -> rtl/
→ Copied FP_Comparator.v -> rtl/
→ Copied Maxpool.v -> rtl/
→ Copied neuron.v -> rtl/
→ Copied FP_Adder.v -> rtl/
→ Copied FP_Multiplier.v -> rtl/
→ Copied Weight_Memory.v -> rtl/
→ Copied Sig_ROM.v -> rtl/
→ Copied relu.v -> rtl/
→ Copied maxFinder.v -> rtl/
→ Copied Conv2D.v -> rtl/
→ Copied FP_Adder.v -> rtl/
→ Copied FP_Multiplier.v -> rtl/
→ Copied ConvMemory.v -> rtl/
→ Copied Conv_SIC.v -> rtl/
→ Copied Conv_MIC.v -> rtl/
→ Copied relu.v -> rtl/
→ Copied Max2D.v -> rtl/
→ Copied FP_Comparator.v -> rtl/
→ Copied Maxpool.v -> rtl/
✓ Wrote Layer_1.v
→ Copied neuron.v -> rtl/
→ Copied FP_Adder.v -> rtl/
→ Copied FP_Multiplier.v -> rtl/
→ Copied Weight_Memory.v -> rtl/
→ Copied Sig_ROM.v -> rtl/
→ Copied relu.v -> rtl/
✓ Wrote Layer_2.v
→ Copied neuron.v -> rtl/
→ C

In [38]:
#!/usr/bin/env python3
"""
package_hw.py

Creates a clean /hw folder and copies:
 - Verilog RTL files (*.v) from /rtl
 - Memory initialization files (*.mif) from /Memory_Files/mif
Excludes all JSON and non-relevant files.
"""

import shutil
from pathlib import Path

# --- Folder paths ---
RTL_DIR = Path("rtl")
MIF_DIR = Path("Memory_Files/mif")
HW_DIR = Path("hw")

def clean_hw_folder():
    """Completely remove and recreate /hw."""
    if HW_DIR.exists():
        print("🧹 Cleaning existing /hw folder ...")
        shutil.rmtree(HW_DIR)
    HW_DIR.mkdir(parents=True, exist_ok=True)

def copy_files(src_dir: Path, dst_dir: Path, extensions):
    """Copy files with specific extensions from src_dir to dst_dir."""
    if not src_dir.exists():
        print(f"⚠ Skipping missing folder: {src_dir}")
        return
    for f in src_dir.iterdir():
        if f.is_file() and f.suffix.lower() in extensions:
            shutil.copy2(f, dst_dir / f.name)
            print(f"→ Copied {f.name}")

def package_hw():
    clean_hw_folder()
    print(f"📦 Packaging RTL and MIF files into {HW_DIR}/")

    # Copy Verilog RTL files
    copy_files(RTL_DIR, HW_DIR, {".v"})

    # Copy .mif files from Memory_Files/mif/
    copy_files(MIF_DIR, HW_DIR, {".mif"})

    print("\n✅ Packaging complete — /hw now contains RTL + MIF files only.")

if __name__ == "__main__":
    package_hw()


🧹 Cleaning existing /hw folder ...
📦 Packaging RTL and MIF files into hw/
→ Copied CNN_top.v
→ Copied Conv2D.v
→ Copied ConvMemory.v
→ Copied ConvolChnl.v
→ Copied Conv_MIC.v
→ Copied Conv_SIC.v
→ Copied FP_Adder.v
→ Copied FP_Comparator.v
→ Copied FP_Multiplier.v
→ Copied ImageBuffer.v
→ Copied ImageBufferChnl.v
→ Copied Layer_1.v
→ Copied Layer_2.v
→ Copied Layer_3.v
→ Copied line_buffer.v
→ Copied Max2D.v
→ Copied maxFinder.v
→ Copied Maxpool.v
→ Copied MaxpoolChnl.v
→ Copied neuron.v
→ Copied Pixel_Buffer.v
→ Copied relu.v
→ Copied Sig_ROM.v
→ Copied Weight_Memory.v
→ Copied b_1_0.mif
→ Copied b_1_1.mif
→ Copied b_1_10.mif
→ Copied b_1_11.mif
→ Copied b_1_12.mif
→ Copied b_1_13.mif
→ Copied b_1_14.mif
→ Copied b_1_15.mif
→ Copied b_1_16.mif
→ Copied b_1_17.mif
→ Copied b_1_18.mif
→ Copied b_1_19.mif
→ Copied b_1_2.mif
→ Copied b_1_20.mif
→ Copied b_1_21.mif
→ Copied b_1_22.mif
→ Copied b_1_23.mif
→ Copied b_1_24.mif
→ Copied b_1_25.mif
→ Copied b_1_26.mif
→ Copied b_1_27.mif
→ Copi